## Packages needed

In [1]:
import datetime as dt
import geopandas as gpd
import os
import fiona
import rasterio.mask
from rasterio.fill import fillnodata
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import rasterio
import ogr
import warnings
import json
import pandas as pd
from earthpy import clip
from shapely.geometry import JOIN_STYLE

import scipy.spatial
from pathlib import Path
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

## Clip raster

In [3]:
def clipRaster(raster, polygon):
    kwargs = {'dstNodata':0}
    warped = gdal.Warp('', raster, cutlineDSName=polygon, format ='MEM', **kwargs)
    return warped

## Clip and save raster

In [1]:
def clip_and_save_raster(raster_path, mask_path, crs, workspace):
    with fiona.open(mask_path, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    with rasterio.open(raster_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_image[out_image<0] = np.nan
        mask = (out_image!=0)
        out_image = fillnodata(out_image, mask)
        out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "crs": crs})
    
    out_meta.update(compress = 'lzw')
    
    with rasterio.open(workspace + r"/elecPop.tif", "w", **out_meta) as dest:
        dest.write(out_image)
    out = rasterio.open(workspace + r"/elecPop.tif")
        
    return out

## Resample raster

In [1]:
def resampleRaster(raster, xRes, yRes):
    kwargs = {'xRes': xRes, 'yRes': yRes, 'noData':'0'} 
    resampled = gdal.Translate('',raster, format ='MEM', **kwargs)
    return resampled

## Reclassify raster

In [1]:
def reclassifyRasters(file):    
    driver = gdal.GetDriverByName("MEM")
    band = file.GetRasterBand(1)
    lista = band.ReadAsArray()

    lista[np.where((0 < lista) & (lista < 999999999))] = 1

    file2 = driver.Create('', file.RasterXSize , file.RasterYSize , 1)
    file2.GetRasterBand(1).WriteArray(lista)

    proj = file.GetProjection()
    georef = file.GetGeoTransform()
    file2.SetProjection(proj)
    file2.SetGeoTransform(georef)
    file2.FlushCache()
    
    return file2

## Convert raster to polygon

In [4]:
def toPolygon(Raster, opt, workspace):
    band = Raster.GetRasterBand(1)
    bandArray = band.ReadAsArray()
    
    if opt == 1:
        outShapefile = workspace + r"\clusters"
    else:
        outShapefile = workspace + r"\NTLArea"
    
    driver = ogr.GetDriverByName("ESRI Shapefile")
    if os.path.exists(outShapefile+".shp"):
        driver.DeleteDataSource(outShapefile+".shp")
    outDatasource = driver.CreateDataSource(outShapefile+ ".shp")
    
    outLayer = outDatasource.CreateLayer(outShapefile+ ".shp", srs=None)
    newField = ogr.FieldDefn('PLACEHOLDE', ogr.OFTInteger)
    outLayer.CreateField(newField)
    gdal.Polygonize(band, None, outLayer, 0, [], callback=None )
    outDatasource.Destroy()
    sourceRaster = None
    
    if opt == 2:
        NTLArea=gpd.read_file(workspace + r"/NTLArea.shp")
        clean = NTLArea[NTLArea.PLACEHOLDE != 0]
        clean_b = clean.buffer(0)
        clean_b.crs = {'init' :'epsg:4326'}
        clean_b.to_file(workspace + r"/NTLArea.shp")  

## Save memory raster


In [1]:
def saveRaster(raster, workspace):
    #Compressing and saving a memory raster layer
    kwargs = {'creationOptions': ['COMPRESS=LZW']}
    gdal.Warp(workspace + r"\elecPop.tif", raster, **kwargs)
    elecPop = rasterio.open(workspace + r"\elecPop.tif")    
    return elecPop

## Buffering

In [5]:
def buffer(polygons, workspace):
    #Adds a buffer and dissolves all of the clusters. Convex hull smooths out edges of settlements
    clean = polygons[polygons.PLACEHOLDE != 0]
    buffered = clean.buffer(0.001)
    convex_hull = buffered.convex_hull
    envgdf = gpd.GeoDataFrame(gpd.GeoSeries(convex_hull))
    envgdf = envgdf.rename(columns={0:'geometry'}).set_geometry('geometry')
    envgdf["fid"] = 1
    dissolved=envgdf.dissolve(by="fid")
    return dissolved

## Convert multipart ot singlepart

In [1]:
def multi2single(gpdf, workspace):
    #Convert multipart polygons to singlepart polygons
    gpdf_singlepoly = gpdf[gpdf.geometry.type == 'Polygon']
    gpdf_multipoly = gpdf[gpdf.geometry.type == 'MultiPolygon']

    for i, row in gpdf_multipoly.iterrows():
        Series_geometries = pd.Series(row.geometry)
        df = pd.concat([gpd.GeoDataFrame(row, crs=gpdf_multipoly.crs).T]*len(Series_geometries), ignore_index=True)
        df['geometry']  = Series_geometries
        gpdf_singlepoly = pd.concat([gpdf_singlepoly, df])

    gpdf_singlepoly.reset_index(inplace=True, drop=True)
    return gpdf_singlepoly

## Dissolve and split

In [7]:
def dissolveandsplit(inputfile, crs, filename_admin, country_name):
    #To make sure all geometries are valid
    eps = 0.001
    dissolved = inputfile.buffer(eps, 1, join_style=JOIN_STYLE.mitre).buffer(-eps, 1, join_style=JOIN_STYLE.mitre)
    envgdf = gpd.GeoDataFrame(gpd.GeoSeries(dissolved))
    envgdf = envgdf.rename(columns={0:'geometry'}).set_geometry('geometry')

    #For the splitting later
    admin = gpd.read_file(filename_admin)
    lines = admin.boundary
    buffered_lines = lines.buffer(0.000000001)
    envgdf2 = gpd.GeoDataFrame(gpd.GeoSeries(buffered_lines))
    envgdf2 = envgdf2.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    #Split and add ids to all rows
    clusters = gpd.overlay(envgdf, envgdf2, how='difference')
    clusters = clusters["geometry"] 
    clusters = gpd.GeoDataFrame(gpd.GeoSeries(clusters))
    clusters = clusters.rename(columns={0:'geometry'}).set_geometry('geometry')
    clusters['id'] = np.arange(len(clusters))
    
    #Reproject and add country name and cluster Area
    clusters.crs = {'init' :'epsg:4326'}
    clusters_proj = clusters.to_crs({ 'init': crs})
    clusters_proj["GridCellAr"] = clusters_proj.area/1000000
    clusters_proj["Country"] = country_name
    clusters = clusters_proj.to_crs({ 'init': 'epsg:4326'})
    return clusters

## Populating clusters

In [8]:
def populatingClusters(clusters,raster,column,method):
    clusters = zonal_stats(
    clusters,
    raster.name,
    stats=[method],
    prefix=column, geojson_out=True, all_touched=True)
    
    return clusters

## Finalizing clusters

In [9]:
def finalizing_clusters(clusters, workspace):
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(clusters)}
        dst.write(json.dumps(collection))
  
    clusters = gpd.read_file(output)
    clusters.fillna(0, inplace=True)
    os.remove(output)
    
    clusters = clusters.rename(columns={"popsum": "Pop"})
    clusters = clusters.rename(columns={"NTLmax": "NightLight"})
    clusters = clusters.rename(columns={"area": "GridCellArea"})
    clusters = clusters.rename(columns={"ElecPopsum": "ElecPop"})
    
    clusters.to_file(workspace + r"\clusters.shp")

    dir_name = workspace
    test = os.listdir(dir_name)

    for item in test:
        if item.endswith(".tif") or item.startswith("NTLArea") or item.startswith("bufferedlines"):
            os.remove(os.path.join(dir_name, item))